
# **Gurobi 4: Generación de Reporte de Solución y Análisis de Recursos**  

## **Introducción**  
En esta práctica, aprenderás a generar un reporte de solución basado en un modelo de optimización.  
Usaremos un **problema de optimización de producción en una panadería**, donde se busca **maximizar la utilidad** considerando costos y restricciones.  

---  
    


## **Descripción del Problema: Producción Óptima en una Panadería** 🥖🥐🍞  

Una panadería produce **tres tipos de pan**:  
1. **Pan Blanco**  
2. **Pan Integral**  
3. **Croissants**  

Cada producto requiere **harina, azúcar y mantequilla** para su producción y toma cierto **tiempo de horneado** en el horno.  
El objetivo es determinar cuántas unidades de cada pan fabricar para **maximizar la utilidad total**.  

📌 **Datos del problema:**  

- **Recursos disponibles:**  
  - Harina: **600 kg**  
  - Azúcar: **200 kg**  
  - Mantequilla: **150 kg**  
  - Tiempo disponible en el horno: **500 horas**  

- **Requerimientos de producción (por unidad de producto):**  

| Producto     | Harina (kg) | Azúcar (kg) | Mantequilla (kg) | Tiempo Horno (h) | Precio de Venta (\$) | Costo de Producción (\$) |
|-------------|------------|------------|-----------------|----------------|----------------|----------------|
| Pan Blanco  | 2          | 1          | 0.5             | 1.5            | 50             | 20             |
| Pan Integral| 3          | 1.5        | 0.7             | 2.0            | 60             | 25             |
| Croissants  | 4          | 2          | 1.5             | 3.5            | 100            | 50             |

📌 **Función objetivo:**  
Maximizar la utilidad total:  

$$
\max Z = \sum_{i=1}^{n} (P_i - C_i) x_i
$$  

Donde:  
- $P_i $ es el precio de venta del producto $ i $.  
- $ C_i $ es el costo de producción del producto $ i$.  
- $ x_i $ es la cantidad de unidades a fabricar de cada producto.  

📌 **Restricciones:**  
1. **Disponibilidad de recursos:**  
   - Uso total de harina ≤ 600 kg  
   - Uso total de azúcar ≤ 200 kg  
   - Uso total de mantequilla ≤ 150 kg  
   - Uso total de tiempo en el horno ≤ 500 horas  
2. **No podemos fabricar cantidades negativas** (restricción de no negatividad).  

---  
    

## **Paso 1: Carga de Datos desde un Archivo CSV o Excel**

In [ ]:

# Importar librerías necesarias
import pandas as pd

# Cargar datos desde CSV
df = pd.read_csv("produccion_panaderia.csv")

# Mostrar los datos cargados
print("Datos cargados:")
print(df)


## **Paso 2: Definir el Modelo de Optimización en Gurobi**

In [ ]:
# Descargar e instalar Gurobi desde la fuente oficial
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y python3.10
!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
!pip install gurobipy

# Importar la biblioteca
import gurobipy as gp
from gurobipy import GRB

print("Gurobi instalado y listo para usarse.")

In [ ]:
# Crear el modelo de optimización
modelo = gp.Model("Produccion_Panaderia")

# Crear variables de decisión (cuántas unidades fabricar de cada pan)
x = modelo.addVars(df["Producto"], vtype=GRB.CONTINUOUS, lb=0, name="x")

# Definir la función objetivo (maximizar la utilidad total)
modelo.setObjective(
    gp.quicksum((df.loc[df["Producto"] == p, "Precio Venta"].values[0] -
                 df.loc[df["Producto"] == p, "Costo Producción"].values[0]) * x[p]
                for p in df["Producto"]), GRB.MAXIMIZE
)

# Restricciones de disponibilidad de recursos

################################################################################
#BORRAR ESTA PARTE PARA QUE EL ALUMNO LA DESAROLLE

modelo.addConstr(
    gp.quicksum(df.loc[df["Producto"] == p, "Harina"].values[0] * x[p]
                for p in df["Producto"]) <= 600, "Harina"
)
modelo.addConstr(
    gp.quicksum(df.loc[df["Producto"] == p, "Azúcar"].values[0] * x[p]
                for p in df["Producto"]) <= 200, "Azúcar"
)
modelo.addConstr(
    gp.quicksum(df.loc[df["Producto"] == p, "Mantequilla"].values[0] * x[p]
                for p in df["Producto"]) <= 150, "Mantequilla"
)
modelo.addConstr(
    gp.quicksum(df.loc[df["Producto"] == p, "Tiempo Horno"].values[0] * x[p]
                for p in df["Producto"]) <= 500, "Tiempo_Horno"
)

################################################################################


# Resolver el modelo
modelo.optimize()


## **Paso 3: Generar el Reporte de Solución y Uso de Recursos**

In [ ]:
################################################################################
#BORRAR ESTA PARTE PARA QUE EL ALUMNO LA DESAROLLE

# Calcular el uso de cada recurso con la solución óptima
harina_usada = sum(df.loc[df["Producto"] == p, "Harina"].values[0] * x[p].x for p in df["Producto"])
azucar_usada = sum(df.loc[df["Producto"] == p, "Azúcar"].values[0] * x[p].x for p in df["Producto"])
mantequilla_usada = sum(df.loc[df["Producto"] == p, "Mantequilla"].values[0] * x[p].x for p in df["Producto"])
tiempo_usado = sum(df.loc[df["Producto"] == p, "Tiempo Horno"].values[0] * x[p].x for p in df["Producto"])

# Mostrar el reporte de solución
print(" **Reporte de Solución**")
print(f"Harina utilizada: {harina_usada:.2f} kg | Harina sobrante: {600 - harina_usada:.2f} kg")
print(f"Azúcar utilizada: {azucar_usada:.2f} kg | Azúcar sobrante: {200 - azucar_usada:.2f} kg")
print(f"Mantequilla utilizada: {mantequilla_usada:.2f} kg | Mantequilla sobrante: {150 - mantequilla_usada:.2f} kg")
print(f"Tiempo horno utilizado: {tiempo_usado:.2f} horas | Tiempo sobrante: {500 - tiempo_usado:.2f} horas")
################################################################################
